### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load
school_data_to_load = "schools_complete.csv"
student_data_to_load = "students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [9]:
# total number of schools
school_count = len(school_data['School ID'].unique())

# total number of students
student_count = len(student_data['Student ID'].unique())

# total budget
total_budget = school_data['budget'].sum()

# average math score
math_average = student_data['math_score'].mean()

# average reading score
reading_average = student_data['reading_score'].mean()

# percent passing math
math_cond = student_data['math_score'] >= 70
math_pass_count = len(student_data['math_score'][math_cond])
math_pass_percentage = math_pass_count / student_count * 100

# percent passing reading
reading_cond = student_data['reading_score'] >= 70
reading_pass_count = len(student_data['reading_score'][reading_cond])
reading_pass_percentage = reading_pass_count / student_count * 100

# total percent passing
total_pass_percentage = (math_pass_percentage + reading_pass_percentage) / 2

In [10]:
# district summary df

district_summary_df = pd.DataFrame({'Total Schools': [school_count],
                           'Total Students': [student_count],
                           'Total Budget': [total_budget],
                           'Average Math Score': [math_average],
                           'Average Reading Score': [math_average],
                           '% Passing Math': [math_pass_percentage],
                           '% Passing Reading': [reading_pass_percentage],
                           '% Overall Passing Rate': [total_pass_percentage]})

# clean up values
district_summary_df['Total Students']=district_summary_df['Total Students'].map("{:,.0f}".format)
district_summary_df['Total Budget']=district_summary_df['Total Budget'].map("${:,.2f}".format)

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,78.985371,74.980853,85.805463,80.393158


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [11]:
# merge the two dfs together

merged_df = pd.merge(school_data, student_data, on='school_name')

# create columns indicating whether the student has a math/reading passing score and replace with 1/0 values
merged_df['Math Passing Score'] = math_cond
merged_df['Math Passing Score'] = merged_df['Math Passing Score'].replace({True: 1,
                                                                           False: 0})
merged_df['Reading Passing Score'] = reading_cond
merged_df['Reading Passing Score'] = merged_df['Reading Passing Score'].replace({True: 1,
                                                                                 False: 0})
# group the data by school
grouped_df = merged_df.groupby('school_name')

# number of students per school
grouped_student_count = grouped_df['student_name'].count()

# budget per student, which is that school's budget divided by number of students
per_student_budget = grouped_df['budget'].max() / grouped_student_count

# average math and reading scores
grouped_math_average = grouped_df['math_score'].mean()
grouped_reading_average = grouped_df['reading_score'].mean()

# percent passing math
grouped_student_count = grouped_df['Student ID'].count()    # use unique ID as proxy for # of students
grouped_math_pass_sum = grouped_df['Math Passing Score'].sum()
math_pass_percentage = grouped_math_pass_sum / grouped_student_count * 100

# percent passing reading
grouped_reading_pass_sum = grouped_df['Reading Passing Score'].sum()
reading_pass_percentage = grouped_reading_pass_sum / grouped_student_count * 100

# overall Passing Rate
overall_pass_percentage = (math_pass_percentage + reading_pass_percentage) / 2

In [12]:
# create school summary df
school_summary_df = pd.DataFrame({
                                  'Total Students': grouped_student_count,
                                  'Per Student Budget': per_student_budget,
                                  'Average Math Score': grouped_math_average,
                                 'Average Reading Score': grouped_reading_average,
                                 '% Passing Math': math_pass_percentage,
                                 '% Passing Reading': reading_pass_percentage,
                                 '% Overall Passing Rate': overall_pass_percentage})


# merge information we already had in the original school data frame
school_data_new_index = school_data.set_index('school_name')
school_data_merge = school_data_new_index[['type', 'budget']]
school_summary_df = pd.merge(school_summary_df, school_data_merge, left_index=True, right_index=True)

# Rename columns

school_summary_df = school_summary_df.rename(columns={"type": "Type",
                                                     'budget': "Total Budget"})

# Reorder columns

school_summary_df = school_summary_df[['Type',
                                       'Total Students', 
                                       'Total Budget',
                                       'Per Student Budget', 
                                       'Average Math Score', 
                                       'Average Reading Score',
                                       '% Passing Math', 
                                       '% Passing Reading', 
                                       '% Overall Passing Rate']]

# clean up a copy, because  we will need this df later on
school_summary_df_cleaned = school_summary_df.copy()

# clean up values
school_summary_df_cleaned['Total Students']=school_summary_df_cleaned['Total Students'].map("{:,.0f}".format)
school_summary_df_cleaned['Total Budget']=school_summary_df_cleaned['Total Budget'].map("${:,.2f}".format)
school_summary_df_cleaned['Per Student Budget']=school_summary_df_cleaned['Per Student Budget'].map("${:,.2f}".format)

school_summary_df_cleaned

,Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [13]:
# five top performing schools

top_performing =  school_summary_df_cleaned.sort_values('% Overall Passing Rate', ascending = False).head(5)

top_performing

,Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [14]:
# five bottom performing schools

bottom_performing = school_summary_df_cleaned.sort_values('% Overall Passing Rate', ascending = True).head(5)

bottom_performing

,Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [15]:
# set school name as index so we can merge all grades together on school
merged_df_index = merged_df.set_index('school_name')

# 9th grade math scores
cond_9th = merged_df_index['grade'] == '9th'
math_9th = merged_df_index['math_score'][cond_9th][1:]
math_9th = math_9th.rename(columns={'math_score': "9th"})
math_9th_group = math_9th.groupby('school_name').mean()

# 10th grade math scores
cond_10th = merged_df_index['grade'] == '10th'
math_10th = merged_df_index['math_score'][cond_10th][1:]
math_10th = math_10th.rename(columns={'math_score': "10th"})
math_10th_group = math_10th.groupby('school_name').mean()

# 11th grade math scores
cond_11th = merged_df_index['grade'] == '11th'
math_11th = merged_df_index['math_score'][cond_11th][1:]
math_11th = math_11th.rename(columns={'math_score': "11th"})
math_11th_group = math_11th.groupby('school_name').mean()

# 12th grade math scores
cond_12th = merged_df_index['grade'] == '12th'
math_12th = merged_df_index['math_score'][cond_12th][1:]
math_12th = math_12th.rename(columns={'math_score': "12th"})
math_12th_group = math_12th.groupby('school_name').mean()

math_by_grade = pd.DataFrame({'9th': math_9th_group,
             '10th': math_10th_group,
             '11th': math_11th_group,
             '12th': math_12th_group})

math_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.024911,75.894256,76.441667,77.253425
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [16]:
# 9th grade reading scores
cond_9th = merged_df_index['grade'] == '9th'
reading_9th = merged_df_index['reading_score'][cond_9th][1:]
reading_9th = reading_9th.rename(columns={'reading_score': "9th"})
reading_9th_group = reading_9th.groupby('school_name').mean()

# 10th grade reading scores
cond_10th = merged_df_index['grade'] == '10th'
reading_10th = merged_df_index['reading_score'][cond_10th][1:]
reading_10th = reading_10th.rename(columns={'reading_score': "10th"})
reading_10th_group = reading_10th.groupby('school_name').mean()

# 11th grade reading scores
cond_11th = merged_df_index['grade'] == '11th'
reading_11th = merged_df_index['reading_score'][cond_11th][1:]
reading_11th = reading_11th.rename(columns={'reading_score': "11th"})
reading_11th_group = reading_11th.groupby('school_name').mean()

# 12th grade reading scores
cond_12th = merged_df_index['grade'] == '12th'
reading_12th = merged_df_index['reading_score'][cond_12th][1:]
reading_12th = reading_12th.rename(columns={'reading_score': "12th"})
reading_12th_group = reading_12th.groupby('school_name').mean()

reading_by_grade = pd.DataFrame({'9th': reading_9th_group,
             '10th': reading_10th_group,
             '11th': reading_11th_group,
             '12th': reading_12th_group})

reading_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.308422,81.494778,81.416667,80.282534
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [17]:
# spending bins
spending_bins = [0, 585, 615, 645, 675]
spending_group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

# copy school summary df
spending_df = school_summary_df.copy()

# create column that assigns spending binis
spending_df['Spending Category'] = pd.cut(school_summary_df['Per Student Budget'], spending_bins, labels = spending_group_names, include_lowest=True)

# group by spending and take the mean of the grouped columns
grouped_spending_df = spending_df.groupby('Spending Category').mean()

# reorder columns

grouped_spending_df = grouped_spending_df[['Average Math Score',
                                       'Average Reading Score',
                                       '% Passing Math', 
                                       '% Passing Reading', 
                                       '% Overall Passing Rate']]

grouped_spending_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Category,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [18]:
# school size bins
size_bins = [0, 1000, 2000, 5000]
size_group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# copy school summary df
size_df = school_summary_df.copy()

# create column that assigns spending binis
size_df['School Size'] = pd.cut(school_summary_df['Total Students'], size_bins, labels=size_group_names, include_lowest=True)

# group by spending and take the mean of the grouped columns
grouped_size_df = size_df.groupby('School Size').mean()

# reorder columns
grouped_size_df = grouped_size_df[['Average Math Score',
                                       'Average Reading Score',
                                       '% Passing Math', 
                                       '% Passing Reading', 
                                       '% Overall Passing Rate']]

grouped_size_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [19]:
# school type bins
type_bins = ['charter', 'district']
type_group_names = ['Charter', 'District']

# copy school summary df
type_df = school_summary_df.copy()

# group by spending and take the mean of the grouped columns
grouped_type_df = size_df.groupby('Type').mean()

# reorder columns
grouped_type_df = grouped_type_df[['Average Math Score',
                                       'Average Reading Score',
                                       '% Passing Math', 
                                       '% Passing Reading', 
                                       '% Overall Passing Rate']]

grouped_type_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


### Two Observable Trends

Based on the tables above:

1. From the table grouping the data by school type (charter vs. district), we see that charter schools perform better across the board based on the math and reading scores and pass rates.
2. From the table grouping the data by school size, small and medium schools perform about the same, while large schools perform worse. It seems that schools that have 2000 or fewer students perform well and then as the school reaches 2000 students they perform less optimally.